In [19]:
import pickle as pkl
import json
with open('concepts/CARS/gpt_res_generation.pkl', 'rb') as fp:
    gpt_res = pkl.load(file=fp)

In [21]:
processed = dict()
for res in gpt_res:
    class_name = res['class_name']
    concepts = json.loads(res['response'])
    processed[class_name] = concepts

In [23]:
with open('concepts/CARS/concepts_processed.json', 'w') as fp:
    json.dump(processed, fp=fp)

In [2]:
import os
import torch
from torch import nn
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.datasets import StanfordCars

from clipseg_model import CLIPSeg
from data.cub_parts_dataset import CUBPartsDataset, collate_fn

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def collate_fn(batch):
    images, labels = list(zip(*batch))
    return images, torch.tensor(labels)
dataset = StanfordCars(root='datasets/CARS', split='test', download=True)
dataloader_test = DataLoader(dataset=dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)
dataloader_test_iter = iter(dataloader_test)

In [6]:
for c in dataset.classes:
    print(c)

AM General Hummer SUV 2000
Acura RL Sedan 2012
Acura TL Sedan 2012
Acura TL Type-S 2008
Acura TSX Sedan 2012
Acura Integra Type R 2001
Acura ZDX Hatchback 2012
Aston Martin V8 Vantage Convertible 2012
Aston Martin V8 Vantage Coupe 2012
Aston Martin Virage Convertible 2012
Aston Martin Virage Coupe 2012
Audi RS 4 Convertible 2008
Audi A5 Coupe 2012
Audi TTS Coupe 2012
Audi R8 Coupe 2012
Audi V8 Sedan 1994
Audi 100 Sedan 1994
Audi 100 Wagon 1994
Audi TT Hatchback 2011
Audi S6 Sedan 2011
Audi S5 Convertible 2012
Audi S5 Coupe 2012
Audi S4 Sedan 2012
Audi S4 Sedan 2007
Audi TT RS Coupe 2012
BMW ActiveHybrid 5 Sedan 2012
BMW 1 Series Convertible 2012
BMW 1 Series Coupe 2012
BMW 3 Series Sedan 2012
BMW 3 Series Wagon 2012
BMW 6 Series Convertible 2007
BMW X5 SUV 2007
BMW X6 SUV 2012
BMW M3 Coupe 2012
BMW M5 Sedan 2010
BMW M6 Convertible 2010
BMW X3 SUV 2012
BMW Z4 Convertible 2012
Bentley Continental Supersports Conv. Convertible 2012
Bentley Arnage Sedan 2009
Bentley Mulsanne Sedan 2011
Ben

In [21]:
from clipseg_train import load_concepts
state_dict = torch.load('checkpoints/clipseg_pascub_ft.pt')

with open('concepts/CUB/parts.txt') as fp:
    part_texts = fp.read().splitlines()

concept_sets = load_concepts()

model = CLIPSeg(
    part_texts=part_texts,
    concepts_dict=concept_sets,
    meta_category_text='bird',
    k=50,
    state_dict=state_dict
)

<class 'clipseg.configuration_clipseg.CLIPSegConfig'> 512


Some weights of CLIPSegForImageSegmentation were not initialized from the model checkpoint at CIDAS/clipseg-rd64-refined and are newly initialized: ['non_object_embedding', 'text_adapter.fc.0.weight', 'text_adapter.fc.2.weight', 'tunable_linear.weight', 'visual_adapter.fc.0.weight', 'visual_adapter.fc.2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
batch = next(dataloader_test_iter)
batch
model.train()
with torch.no_grad():
    imgs, labels = batch
    imgs = list(imgs)
    results = model(imgs, labels)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_CUDA_mm)

In [32]:
if torch.sum(torch.zeros((100, 100))) == 0:
    print('hah')

hah
